In [28]:
data=[
    ['A','a',5],
    ['A','b',10],
    ['A','c',10],
    ['A','d',5],
    ['B','a',4],
    ['B','b',5],
    ['B','c',4],
    ['B','d',10],
    ['C','a',4],
    ['C','c',10],
    ['C','d',5],
]

In [29]:
train_data = {}


for user, item,record in data:
    train_data.setdefault(user,{})
    train_data[user][item] = record


train_data

{'A': {'a': 5, 'b': 10, 'c': 10, 'd': 5},
 'B': {'a': 4, 'b': 5, 'c': 4, 'd': 10},
 'C': {'a': 4, 'c': 10, 'd': 5}}

In [30]:
# 建立item_user倒排表
item_users = dict()
for u, items in train_data.items():
    for i in items:
        if i not in item_users:
            item_users[i] = set()
        item_users[i].add(u)
item_users

{'a': {'A', 'B', 'C'},
 'b': {'A', 'B'},
 'c': {'A', 'B', 'C'},
 'd': {'A', 'B', 'C'}}

In [31]:
def predict(user, j):
    rating = 0
    cnt = 0
    if j not in item_users:
        return rating
    for i in train_data[user]:
        if i == j:
            continue
        if i not in item_users:
            continue
        common_users = item_users[i] & item_users[j]
        if len(common_users) == 0:
            continue
        dev_ij = 0
        cnt += 1
        for common_user in common_users:
            dev_ij += (train_data[common_user][j] - train_data[common_user][i])
        dev_ij /= len(common_users)
        rating += (dev_ij + train_data[user][i])
    rating /= cnt
    return rating


In [32]:
predict('C','b')

7.5